In [1]:
## Basic stuff
%load_ext autoreload
%autoreload
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
display(HTML("""<style>div.output_area{max-height:10000px;overflow:scroll;}</style>"""))

## Python Version
import sys
print("Python: {0}".format(sys.version))

from combine import combine
from fsUtils import moveFile
from ioUtils import getFile, saveFile
from timeUtils import clock, elapsed
from pandas import DataFrame, Series, read_csv


import datetime as dt
start = dt.datetime.now()
print("Notebook Last Run Initiated: "+str(start))

Python: 3.7.3 (default, Mar 27 2019, 16:54:48) 
[Clang 4.0.1 (tags/RELEASE_401/final)]
Notebook Last Run Initiated: 2019-11-05 21:11:00.498701


In [2]:
oscarsURL = "https://www.imdb.com/oscars/nominations/?ref_=ev_all"
baftasURL = "https://www.imdb.com/awards-central/baftas/?ref_=ev_all"
goldenglobesURL = "https://www.imdb.com/golden-globes/nominations/?ref_=ev_all"
sagURL = "https://www.imdb.com/event/ev0000598/2019/1"
cannesURL = "https://www.imdb.com/event/ev0000147/2019/1"
bodilURL = "https://www.imdb.com/event/ev0000107/2019/1"
torontoURL = "https://www.imdb.com/event/ev0000659/2019/1"
sundanceURL = "https://www.imdb.com/event/ev0000631/2019/1"
soundURL = "https://www.imdb.com/event/ev0000452/2019/1"
teenURL = "https://www.imdb.com/event/ev0000644/2019/1"

In [189]:
from webUtils import getWebData

url      = oscarsURL
savename = "awards/IMDB_oscars_base.p"
#getWebData(base=url, savename=savename, useSafari=False, debug=True)

url      = baftasURL
savename = "awards/IMDB_baftas_base.p"
#getWebData(base=url, savename=savename, useSafari=False, debug=True)

url      = goldenglobesURL
savename = "awards/IMDB_goldenglobes_base.p"
#getWebData(base=url, savename=savename, useSafari=False, debug=True)

url      = sagURL
savename = "awards/IMDB_sag_base.p"
#getWebData(base=url, savename=savename, useSafari=False, debug=True)

url      = cannesURL
savename = "awards/IMDB_cannes_base.p"
#getWebData(base=url, savename=savename, useSafari=False, debug=True)

url      = bodilURL
savename = "awards/IMDB_bodil_base.p"
#getWebData(base=url, savename=savename, useSafari=False, debug=True)

url      = torontoURL
savename = "awards/IMDB_toronto_base.p"
#getWebData(base=url, savename=savename, useSafari=False, debug=True)

url      = sundanceURL
savename = "awards/IMDB_sundance_base.p"
#getWebData(base=url, savename=savename, useSafari=False, debug=True)

url      = soundURL
savename = "awards/IMDB_sound_base.p"
#getWebData(base=url, savename=savename, useSafari=False, debug=True)

url      = teenURL
savename = "awards/IMDB_teen_base.p"
#getWebData(base=url, savename=savename, useSafari=False, debug=True)

# Parse Data

In [190]:
from webUtils import getHTML
#oscarsData = getHTML("awards/IMDB_oscars_base.p")

In [302]:
from time import sleep
import json
from searchUtils import findPatternExt


movieMap = {"baftas": "BAFTA Film Award", "oscars": "Oscar", "sag": "Actor", "teen": "Teen Choice Award", "sound": "Golden Reel Award", "toronto": "People's Choice Award", "sundance": "Grand Jury Prize", "cannes": "Palme d'Or", "bodil": "Bodil"}

def getMovieAndPeople(primaryData, secondaryData):
    movie  = {}
    people = {}
    for pdata in primaryData:
        if pdata[1].startswith("tt"):
            movie.update({pdata[1]: pdata[0]})
        if pdata[1].startswith("nm"):
            people.update({pdata[1]: pdata[0]})
            
    for sdata in secondaryData:
        if sdata[1].startswith("tt"):
            movie.update({sdata[1]: sdata[0]})
        if sdata[1].startswith("nm"):
            people.update({sdata[1]: sdata[0]})
            
    return movie,people


def getPageData(data, awardsName):
    dd = data["nomineesWidgetModel"]['eventEditionSummary']
    awards = dd['awards']
    year   = dd['year']
    
    dataname = movieMap.get(awardsName)
    
    awardsData = {}
    for award in awards:
        for category in award['categories']:
            name     = category["categoryName"]
            nominees = category["nominations"]
            categoryData = {"Winner": [], "Nominees": []}
            for nominee in nominees:
                primary   = nominee['primaryNominees']
                secondary = nominee['secondaryNominees']
                isWinner  = nominee['isWinner']

                
                primaryData   = [(x['name'],x['const']) for x in primary]
                secondaryData = [(x['name'],x['const']) for x in secondary]
                
                movie,people = getMovieAndPeople(primaryData, secondaryData)
                if isWinner is True:
                    categoryData["Winner"].append({"Movie": movie, "People": people})
                else:
                    categoryData["Nominees"].append({"Movie": movie, "People": people})

            awardsData[name] = categoryData
            
    retval = {year: awardsData}
    return retval

def parseOscarsData(awardsName="oscars"):

    awardsData = {}
    files = findPatternExt("awards", pattern="IMDB_{0}_".format(awardsName), ext=".p")
    #print("Found {0} files".format(len(files)))
    for oscarsFile in files:
        if "base.p" in oscarsFile:
            continue
            
        oscarsData = getHTML(oscarsFile)
        for sdata in oscarsData.findAll("script", {"type": "text/javascript"}):

            txt = sdata.text
            pos = txt.find("IMDbReactWidgets.NomineesWidget.push")
            if pos == -1:
                continue
            start = pos + len("IMDbReactWidgets.NomineesWidget.push(")
            end   = txt.rfind("]);") + 1

            txtdata = txt[start:end]
            try:
                data = json.loads(txtdata[len("['center-8-react',"):-1])
            except:
                raise ValueError("Cannot find dictionary in data!!!")

            retval = getPageData(data, awardsName)
            awardsData.update(retval)
    return awardsData
    
    
def getOscarsData(awardsName="oscars"):
    awardsData = {}
    
    oscarsData = getHTML("awards/IMDB_{0}_base.p".format(awardsName))
    for sdata in oscarsData.findAll("script", {"type": "text/javascript"}):

        txt = sdata.text
        pos = txt.find("IMDbReactWidgets.EventHistoryWidget.push")
        if pos == -1:
            continue
        start = pos + len("IMDbReactWidgets.EventHistoryWidget.push(")
        end   = txt.rfind("]);") + 1

        txtdata = txt[start:end]
        try:
            data = json.loads(txtdata[len("['right-1-react',"):-1])
        except:
            raise ValueError("Cannot find dictionary in data!!!")

        eventID = data['eventHistoryWidgetModel']['eventId']
        for event in data['eventHistoryWidgetModel']['eventEditions']:
            #eventID  = event['eventEditionId']
            year     = event['year']

            #url      = "https://www.imdb.com/event/ev0000003/2018/1/
            url      = "https://www.imdb.com/event/{0}/{1}/1/".format(eventID, year)
            savename = "awards/IMDB_{0}_{1}.p".format(awardsName, year)
            try:
                getWebData(base=url, savename=savename, useSafari=False, debug=True)        
                sleep(1.5)
            except:
                break
    
    #retval = getPageData(data)
    #awardsData.update(retval)
    #print(len(awardsData))
    
    print("\n\n\n")

In [303]:
#getOscarsData("teen")
#parseOscarsData("oscars")

{1944: {'Best Actor in a Leading Role': {'Winner': [{'Movie': {'tt0036515': 'Watch on the Rhine'},
     'People': {'nm0510134': 'Paul Lukas'}}],
   'Nominees': [{'Movie': {'tt0035896': 'For Whom the Bell Tolls'},
     'People': {'nm0000011': 'Gary Cooper'}},
    {'Movie': {'tt0034583': 'Casablanca'},
     'People': {'nm0000007': 'Humphrey Bogart'}},
    {'Movie': {'tt0036022': 'The Human Comedy'},
     'People': {'nm0001682': 'Mickey Rooney'}},
    {'Movie': {'tt0036126': 'Madame Curie'},
     'People': {'nm0682074': 'Walter Pidgeon'}}]},
  'Best Actor in a Supporting Role': {'Winner': [{'Movie': {'tt0036172': 'The More the Merrier'},
     'People': {'nm0002013': 'Charles Coburn'}}],
   'Nominees': [{'Movie': {'tt0035896': 'For Whom the Bell Tolls'},
     'People': {'nm0848667': 'Akim Tamiroff'}},
    {'Movie': {'tt0036377': 'The Song of Bernadette'},
     'People': {'nm0001948': 'Charles Bickford'}},
    {'Movie': {'tt0034583': 'Casablanca'},
     'People': {'nm0001647': 'Claude Rains

In [304]:
eventData = {}
for event in movieMap.keys():
    eventData[event] = parseOscarsData(event)

In [305]:
from ioUtils import getFile, saveFile

In [306]:
saveFile(idata=eventData, ifile="awards/events.p", debug=True)

Saving data to awards/events.p
  --> This file is 2.3MB.
Saved data to awards/events.p
  --> This file is 2.3MB.


# Clean Events

In [3]:
eventData = getFile("awards/events.p")

In [4]:
eventData.keys()

dict_keys(['baftas', 'oscars', 'sag', 'teen', 'sound', 'toronto', 'sundance', 'cannes', 'bodil'])

In [5]:
eventOrder = {}
eventOrder['oscars']   = len(eventOrder)
eventOrder['baftas']   = len(eventOrder)
eventOrder['sag']      = len(eventOrder)
eventOrder['toronto']  = len(eventOrder)
eventOrder['cannes']   = len(eventOrder)
eventOrder['sundance'] = len(eventOrder)
eventOrder['bodil']    = len(eventOrder)
eventOrder['teen']     = len(eventOrder)
eventOrder['sound']    = len(eventOrder)
eventOrder

{'oscars': 0,
 'baftas': 1,
 'sag': 2,
 'toronto': 3,
 'cannes': 4,
 'sundance': 5,
 'bodil': 6,
 'teen': 7,
 'sound': 8}

In [6]:
def addFilm(films, film, year, event):
    key     = "{0} [{1}]".format(film, year)
    lastKey = "{0} [{1}]".format(film, year-1)
    nextKey = "{0} [{1}]".format(film, year+1)
    lastlastKey = "{0} [{1}]".format(film, year-2)
    nextnextKey = "{0} [{1}]".format(film, year+2)
    if films.get(lastlastKey) is None and films.get(nextnextKey is None) and films.get(lastKey) is None and films.get(nextKey is None) and films.get(key) is None:
        print("\tFound {0} ==> {1} . Adding it".format(key,eventOrder[event]))
        films[key] = eventOrder[event]
        return films
        
    
    if films.get(key) is None:
        if films.get(lastKey) is not None:
            if eventOrder[event] < films[lastKey]:
                print("\tFound {0} ==> {1}  and {2} ==> {3} . Removing the latter and adding the former".format(key,eventOrder[event],lastKey,films[lastKey]))
                del films[lastKey]
                films[key] = eventOrder[event]
            else:
                print("\tFound {0} ==> {1}  and {2} ==> {3} . Keeping the latter and not adding the former".format(key,eventOrder[event],lastKey,films[lastKey]))                
        elif films.get(lastlastKey) is not None:
            if eventOrder[event] < films[lastlastKey]:
                print("\tFound {0} ==> {1}  and {2} ==> {3} . Removing the latter and adding the former".format(key,eventOrder[event],lastlastKey,films[lastlastKey]))
                del films[lastlastKey]
                films[key] = eventOrder[event]
            else:
                print("\tFound {0} ==> {1}  and {2} ==> {3} . Keeping the latter and not adding the former".format(key,eventOrder[event],lastlastKey,films[lastlastKey]))                
        elif films.get(nextKey) is not None:
            if eventOrder[event] < films[nextKey]:
                print("\tFound {0} ==> {1}  and {2} ==> {3} . Removing the latter and adding the former".format(key,eventOrder[event],nextKey,films[nextKey]))
                del films[nextKey]
                films[key] = eventOrder[event]
            else:
                print("\tFound {0} ==> {1}  and {2} ==> {3} . Keeping the latter and not adding the former".format(key,eventOrder[event],nextKey,films[nextKey]))                
        elif films.get(nextnextKey) is not None:
            if eventOrder[event] < films[nextnextKey]:
                print("\tFound {0} ==> {1}  and {2} ==> {3} . Removing the latter and adding the former".format(key,eventOrder[event],nextnextKey,films[nextnextKey]))
                del films[nextnextKey]
                films[key] = eventOrder[event]
            else:
                print("\tFound {0} ==> {1}  and {2} ==> {3} . Keeping the latter and not adding the former".format(key,eventOrder[event],nextnextKey,films[nextnextKey]))                
        else:
            print("\tFound {0} ==> {1} . Adding it".format(key,eventOrder[event]))
            films[key] = eventOrder[event]
    else:
        if films.get(lastKey) is not None:
            if films[key] < films[lastKey]:
                print("\tFound {0} ==> {1}  and {2} ==> {3} . Removing the latter and keeping the former".format(key,eventOrder[event],lastKey,films[lastKey]))
                del films[lastKey]
            else:
                print("\tFound {0} ==> {1}  and {2} ==> {3} . Removing the former and keeping the latter".format(key,eventOrder[event],lastKey,films[lastKey]))
                del films[key]
        if films.get(lastlastKey) is not None:
            if films[key] < films[lastlastKey]:
                print("\tFound {0} ==> {1}  and {2} ==> {3} . Removing the latter and keeping the former".format(key,eventOrder[event],lastlastKey,films[lastlastKey]))
                del films[lastlastKey]
            else:
                print("\tFound {0} ==> {1}  and {2} ==> {3} . Removing the former and keeping the latter".format(key,eventOrder[event],lastlastKey,films[lastlastKey]))
                del films[key]
        if films.get(nextKey) is not None:
            if films[key] < films[nextKey]:
                print("\tFound {0} ==> {1}  and {2} ==> {3} . Removing the latter and keeping the former".format(key,eventOrder[event],nextKey,films[nextKey]))
                del films[nextKey]
            else:
                print("\tFound {0} ==> {1}  and {2} ==> {3} . Removing the former and keeping the latter".format(key,eventOrder[event],nextKey,films[nextKey]))
                del films[key]
        if films.get(nextnextKey) is not None:
            if films[key] < films[nextnextKey]:
                print("\tFound {0} ==> {1}  and {2} ==> {3} . Removing the latter and keeping the former".format(key,eventOrder[event],nextnextKey,films[nextnextKey]))
                del films[nextnextKey]
            else:
                print("\tFound {0} ==> {1}  and {2} ==> {3} . Removing the former and keeping the latter".format(key,eventOrder[event],nextnextKey,films[nextnextKey]))
                del films[key]
                
    return films

                            
films = {}
stopfilm = "Rick Baker"
from collection import Counter
catCntr = Counter()

for event in eventOrder.keys():
    eventResults = eventData[event]
    for year in sorted(eventResults.keys()):
        yearResults = eventResults[year]
        for category,categoryResults in yearResults.items():
            catCntr[category] += 1
            winner   = categoryResults['Winner']
            nominees = categoryResults['Nominees']
                        
            ## Movie
            for item in winner:
                movies = item["Movie"]
                people = item["People"]
                print(movies)
                for key,movie in movies.items():
                    film = "{0} [{1}]".format(movie, key)
                    films = addFilm(films, film, year, event)
                
            for item in nominees:
                movies = item["Movie"]
                people = item["People"]
                for key,movie in movies.items():
                    film = "{0} [{1}]".format(movie, key)
                    films = addFilm(films, film, year, event)

                
saveFile(idata=films, ifile="awards/films.p", debug=True)

{'tt0019071': 'The Last Command', 'tt0019553': 'The Way of All Flesh'}
	Found The Last Command [tt0019071] [1929] ==> 0 . Adding it
	Found The Way of All Flesh [tt0019553] [1929] ==> 0 . Adding it
	Found The Patent Leather Kid [tt0018253] [1929] ==> 0 . Adding it
	Found The Noose [tt0019217] [1929] ==> 0 . Adding it
{'tt0018379': '7th Heaven', 'tt0019429': 'Street Angel', 'tt0018455': 'Sunrise'}
	Found 7th Heaven [tt0018379] [1929] ==> 0 . Adding it
	Found Street Angel [tt0019429] [1929] ==> 0 . Adding it
	Found Sunrise [tt0018455] [1929] ==> 0 . Adding it
	Found Sadie Thompson [tt0019344] [1929] ==> 0 . Adding it
	Found A Ship Comes In [tt0018389] [1929] ==> 0 . Adding it
{'tt0019451': 'Tempest'}
	Found Tempest [tt0019451] [1929] ==> 0 . Adding it
{'tt0017822': 'The Dove'}
	Found The Dove [tt0017822] [1929] ==> 0 . Adding it
{'tt0018455': 'Sunrise'}
	Found The Devil Dancer [tt0017806] [1929] ==> 0 . Adding it
	Found The Magic Flame [tt0018129] [1929] ==> 0 . Adding it
{'tt0018515': 'T

	Found Dillinger [tt0037644] [1946] ==> 0 . Adding it
	Found Music for Millions [tt0037104] [1946] ==> 0 . Adding it
	Found Salty O'Rourke [tt0038047] [1946] ==> 0 . Adding it
	Found What Next, Corporal Hargrove? [tt0038243] [1946] ==> 0 . Adding it
{'tt0037795': 'The House on 92nd Street'}
	Found The House on 92nd Street [tt0037795] [1946] ==> 0 . Adding it
	Found The Affairs of Susan [tt0037498] [1946] ==> 0 . Adding it
{'tt0037884': 'The Lost Weekend'}
	Found Pride of the Marines [tt0038000] [1946] ==> 0 . Adding it
{}
{}
{'tt0036868': 'The Best Years of Our Lives'}
	Found The Best Years of Our Lives [tt0036868] [1947] ==> 0 . Adding it
	Found The Yearling [tt0039111] [1947] ==> 0 . Adding it
	Found It's a Wonderful Life [tt0038650] [1947] ==> 0 . Adding it
	Found The Jolson Story [tt0038661] [1947] ==> 0 . Adding it
	Found Henry V [tt0036910] [1947] ==> 0 . Adding it
{'tt0036868': 'The Best Years of Our Lives'}
	Found The Green Years [tt0038578] [1947] ==> 0 . Adding it
	Found Noto

	Found Giuseppina [tt0053866] [1961] ==> 0 . Adding it
	Found Beyond Silence [tt0053649] [1961] ==> 0 . Adding it
	Found En by ved navn København [tt0053685] [1961] ==> 0 . Adding it
	Found George Grosz' Interregnum [tt0053858] [1961] ==> 0 . Adding it
	Found Universe [tt0054429] [1961] ==> 0 . Adding it
{'tt0054387': 'The Time Machine'}
	Found The Time Machine [tt0054387] [1961] ==> 0 . Adding it
	Found The Last Voyage [tt0054016] [1961] ==> 0 . Adding it
{'tt0053604': 'The Apartment'}
{'tt0053976': 'The Virgin Spring'}
	Found Kapò [tt0052961] [1961] ==> 0 . Adding it
	Found La vérité [tt0054452] [1961] ==> 0 . Adding it
	Found Macario [tt0054042] [1961] ==> 0 . Adding it
	Found The Ninth Circle [tt0053764] [1961] ==> 0 . Adding it
{'tt0054198': 'Never on Sunday'}
	Found High Time [tt0053912] [1961] ==> 0 . Adding it
{'tt0053804': 'Exodus'}
	Found The Magnificent Seven [tt0054047] [1961] ==> 0 . Adding it
{'tt0054324': 'Song Without End'}
	Found Song Without End [tt0054324] [1961] ==>

{'tt0076918': 'Who Are the DeBolts? and Where Did They Get Nineteen Kids?'}
	Found Who Are the DeBolts? and Where Did They Get Nineteen Kids? [tt0076918] [1978] ==> 0 . Adding it
	Found High Grass Circus [tt0076142] [1978] ==> 0 . Adding it
	Found Homage to Chagall: The Colours of Love [tt0076151] [1978] ==> 0 . Adding it
	Found The Children of Theatre Street [tt0075839] [1978] ==> 0 . Adding it
	Found Union Maids [tt0075372] [1978] ==> 0 . Adding it
{'tt0076105': 'Gravity Is My Enemy'}
	Found Gravity Is My Enemy [tt0076105] [1978] ==> 0 . Adding it
	Found Agueda Martinez: Our People, Our Country [tt0075645] [1978] ==> 0 . Adding it
	Found First Edition [tt0076032] [1978] ==> 0 . Adding it
	Found Of Time, Tombs and Treasures [tt0076488] [1978] ==> 0 . Adding it
	Found The Shetland Experience [tt0076700] [1978] ==> 0 . Adding it
{'tt0076759': 'Star Wars: Episode IV - A New Hope'}
{'tt0076759': 'Star Wars: Episode IV - A New Hope'}
	Found Smokey and the Bandit [tt0076729] [1978] ==> 0 . 

{'tt0105695': 'Unforgiven'}
{'tt0104908': 'Mona Lisa Descending a Staircase'}
	Found Mona Lisa Descending a Staircase [tt0104908] [1993] ==> 0 . Adding it
	Found Adam [tt0101270] [1993] ==> 0 . Adding it
	Found Screen Play [tt0105335] [1993] ==> 0 . Adding it
	Found The Sandman [tt0105311] [1993] ==> 0 . Adding it
	Found Words, Words, Words [tt0105224] [1993] ==> 0 . Adding it
{'tt0105051': 'Omnibus'}
	Found Omnibus [tt0105051] [1993] ==> 0 . Adding it
	Found Contact [tt0104007] [1993] ==> 0 . Adding it
	Found Cruise Control [tt0104032] [1993] ==> 0 . Adding it
	Found Swan Song [tt0105506] [1993] ==> 0 . Adding it
	Found The Lady in Waiting [tt0104669] [1993] ==> 0 . Adding it
{'tt0104691': 'The Last of the Mohicans'}
	Found The Last of the Mohicans [tt0104691] [1993] ==> 0 . Adding it
{'tt0104454': 'Howards End'}
{'tt0104036': 'The Crying Game'}
{}
{'tt0107818': 'Philadelphia'}
	Found Philadelphia [tt0107818] [1994] ==> 0 . Adding it
	Found The Remains of the Day [tt0107943] [1994] ==

...

	Found Saw III [tt0489270] [2007] ==> 7 . Adding it
{'tt0383574': "Pirates of the Caribbean: Dead Man's Chest"}
	Found Spider-Man 3 [tt0413300] [2007] ==> 7  and Spider-Man 3 [tt0413300] [2008] ==> 1 . Keeping the latter and not adding the former
	Found Transformers [tt0418279] [2007] ==> 7  and Transformers [tt0418279] [2008] ==> 0 . Keeping the latter and not adding the former
{'tt0449088': "Pirates of the Caribbean: At World's End"}
	Found Pirates of the Caribbean: At World's End [tt0449088] [2007] ==> 7  and Pirates of the Caribbean: At World's End [tt0449088] [2008] ==> 0 . Keeping the latter and not adding the former
	Found 300 [tt0416449] [2007] ==> 7  and 300 [tt0416449] [2008] ==> 2 . Keeping the latter and not adding the former
	Found Spider-Man 3 [tt0413300] [2007] ==> 7  and Spider-Man 3 [tt0413300] [2008] ==> 1 . Keeping the latter and not adding the former
	Found Transformers [tt0418279] [2007] ==> 7  and Transformers [tt0418279] [2008] ==> 0 . Keeping the latter and not 

	Found Safe Haven [tt1702439] [2013] ==> 7 . Adding it
{'tt1673434': 'The Twilight Saga: Breaking Dawn - Part 2'}
	Found Oz the Great and Powerful [tt1623205] [2013] ==> 7 . Adding it
	Found Iron Man 3 [tt1300854] [2013] ==> 7  and Iron Man 3 [tt1300854] [2014] ==> 0 . Keeping the latter and not adding the former
	Found Oblivion [tt1483013] [2013] ==> 7 . Adding it
{'tt1345836': 'The Dark Knight Rises'}
	Found Iron Man 3 [tt1300854] [2013] ==> 7  and Iron Man 3 [tt1300854] [2014] ==> 0 . Keeping the latter and not adding the former
	Found Total Recall [tt1386703] [2013] ==> 7 . Adding it
{'tt1981677': 'Pitch Perfect'}
{'tt1659337': 'The Perks of Being a Wallflower'}
	Found The Great Gatsby [tt1343092] [2013] ==> 7  and The Great Gatsby [tt1343092] [2014] ==> 0 . Keeping the latter and not adding the former
	Found The Call [tt1911644] [2013] ==> 7 . Adding it
{'tt1673434': 'The Twilight Saga: Breaking Dawn - Part 2'}
	Found Playing for Keeps [tt1540128] [2013] ==> 7 . Adding it
{'tt1673

	Found Star [tt4941240] [2019] ==> 7  and Star [tt4941240] [2017] ==> 7 . Keeping the latter and not adding the former
	Found The Resident [tt6483832] [2019] ==> 7  and The Resident [tt6483832] [2018] ==> 7 . Keeping the latter and not adding the former
{'tt4145054': 'Shadowhunters'}
	Found The 100 [tt2661044] [2019] ==> 7  and The 100 [tt2661044] [2018] ==> 7 . Keeping the latter and not adding the former
{'tt3749900': 'Gotham'}
	Found The Flash [tt3107288] [2019] ==> 7  and The Flash [tt3107288] [2018] ==> 7 . Keeping the latter and not adding the former
	Found The Good Place [tt4955642] [2019] ==> 7  and The Good Place [tt4955642] [2018] ==> 7 . Keeping the latter and not adding the former
{'tt4126476': 'After'}
{'tt0046534': 'The War of the Worlds'}
{'tt0047573': 'Them!'}
{'tt0047577': 'This Island Earth'}
	Found This Island Earth [tt0047577] [1956] ==> 8 . Adding it
{'tt0049169': 'Earth vs. the Flying Saucers'}
	Found Earth vs. the Flying Saucers [tt0049169] [1957] ==> 8 . Adding 

	Found CSI: Miami [tt0313043] [2006] ==> 8  and CSI: Miami [tt0313043] [2004] ==> 8 . Keeping the latter and not adding the former
	Found ER [tt0108757] [2006] ==> 8  and ER [tt0108757] [2004] ==> 7 . Keeping the latter and not adding the former
{'tt0412175': 'Medium'}
	Found Battlestar Galactica [tt0407362] [2006] ==> 8 . Adding it
	Found CSI: Crime Scene Investigation [tt0247082] [2006] ==> 8  and CSI: Crime Scene Investigation [tt0247082] [2005] ==> 2 . Keeping the latter and not adding the former
	Found Cold Case [tt0368479] [2006] ==> 8  and Cold Case [tt0368479] [2005] ==> 8 . Keeping the latter and not adding the former
	Found Desperate Housewives [tt0410975] [2006] ==> 8  and Desperate Housewives [tt0410975] [2005] ==> 2 . Keeping the latter and not adding the former
	Found Six Feet Under [tt0248654] [2006] ==> 8  and Six Feet Under [tt0248654] [2005] ==> 2 . Keeping the latter and not adding the former
{'tt0108757': 'ER'}
	Found ER [tt0108757] [2006] ==> 8  and ER [tt0108757] 

  --> This file is 268.9kB.
Saved data to awards/films.p
  --> This file is 268.9kB.


In [7]:
Series(films)

The Last Command [tt0019071] [1929]                              0
The Way of All Flesh [tt0019553] [1929]                          0
The Patent Leather Kid [tt0018253] [1929]                        0
The Noose [tt0019217] [1929]                                     0
7th Heaven [tt0018379] [1929]                                    0
Street Angel [tt0019429] [1929]                                  0
Sunrise [tt0018455] [1929]                                       0
Sadie Thompson [tt0019344] [1929]                                0
A Ship Comes In [tt0018389] [1929]                               0
Tempest [tt0019451] [1929]                                       0
The Dove [tt0017822] [1929]                                      0
The Devil Dancer [tt0017806] [1929]                              0
The Magic Flame [tt0018129] [1929]                               0
Two Arabian Knights [tt0018515] [1929]                           0
Speedy [tt0019412] [1929]                                     